# Social networks

## Want to really learn about this?

http://networksciencebook.com/


## First, lets make a small social network

### step 1: make a set of nodes (people)

In [6]:
import random
from math import log

In [16]:
n = 10
p = 1.1*(log(n)/n)

In [13]:
import random

with open("../datasets/first_names.txt", "r") as f:
    names = f.read().splitlines()

names = sorted(random.sample(names, k=n))

In [207]:
print(names)

['Adelia', 'Alexi', 'Alfonzo', 'Amish', 'Archie', 'Arley', 'Arlie', 'Aser', 'Ayonna', 'Benjamen', 'Branson', 'Brenner', 'Brittiney', 'Carine', 'Cezar', 'Charita', 'Cherylann', 'Chrystine', 'Courtnay', 'Dani', 'Dejan', 'Dung', 'Elnora', 'Elzabeth', 'Garth', 'Heathr', 'Hogan', 'Hunter', 'Ibeth', 'Isidore', 'Jazzmon', 'Jeanell', 'Jermika', 'Jermine', 'Jeston', 'Johndaniel', 'Jonika', 'Karisha', 'Katharine', 'Kaylor', 'Kezia', 'Kiah', 'Kizzie', 'Krystyn', 'Lakeita', 'Laria', 'Lasheika', 'Lashonda', 'Lavonte', 'Laysa', 'Leigh', 'Mareena', 'Marjorie', 'Marleny', 'Marnell', 'Marrissa', 'Mazin', 'Mccall', 'Mickaela', 'Mirranda', 'Mystique', 'Naim', 'Nashea', 'Natassja', 'Nikolaus', 'Noha', 'Patric', 'Pearlie', 'Porschia', 'Racquel', 'Raina', 'Randalyn', 'Raymund', 'Rhiannon', 'Rindy', 'Romney', 'Sabreen', 'Sabreena', 'Shaneice', 'Shaniqua', 'Shareen', 'Shawta', 'Shikita', 'Simone', 'Susanne', 'Tahara', 'Tahra', 'Tamilia', 'Ted', 'Tinesha', 'Tynetta', 'Valarie', 'Valine', 'Venetta', 'Vernonica'

### step 2: make edges

Here we will define an Erdős-Rényi graph (ER graph) random graph, where each pair of people has some probability $p$ of having a connection.

In [18]:
edges = dict(zip(names, [[] for i in range(n)]))

for i in range(n-1):
    for j in range(i+1,n):
        if random.random() < p:
            # edge exists, add symmetrically
            edges[names[i]].append(names[j])
            edges[names[j]] += [names[i]]

In [19]:
for k,v in edges.items():
    print(f'{k:10}', v)

Cade       ['Moises']
Camry      ['Christiaan']
Christiaan ['Camry', 'Joseangel', 'Moises', 'Sameerah', 'Shanise']
Joseangel  ['Christiaan', 'Moises', 'Qunisha', 'Sameerah']
Kaetlyn    ['Qunisha', 'Ronda']
Moises     ['Cade', 'Christiaan', 'Joseangel', 'Qunisha', 'Shanise']
Qunisha    ['Joseangel', 'Kaetlyn', 'Moises', 'Ronda']
Ronda      ['Kaetlyn', 'Qunisha']
Sameerah   ['Christiaan', 'Joseangel']
Shanise    ['Christiaan', 'Moises']


## Basic effects

### Your friends have more friends than you.


In [20]:
def mean(lst):  # could also `from statistics import mean`, but this is a bit faster, and thats all we need
    return(sum(lst)/len(lst))

nff_m_nf = []
for person,friends in edges.items():
    n_friends = len(friends)
    print(f'{person:10} has {n_friends} friends ', end='')
    if(n_friends > 0):
        n_friends_per_friend = [len(edges[fr]) for fr in friends]
        print(f'with an average of {mean(n_friends_per_friend):.2f} friends each', end='')
        nff_m_nf.append(mean(n_friends_per_friend) - n_friends)
    print('')
            

print(f'\nPeople have {mean(nff_m_nf):.2f} fewer friends than the average number of friends their friends have')

Cade       has 1 friends with an average of 5.00 friends each
Camry      has 1 friends with an average of 5.00 friends each
Christiaan has 5 friends with an average of 2.80 friends each
Joseangel  has 4 friends with an average of 4.00 friends each
Kaetlyn    has 2 friends with an average of 3.00 friends each
Moises     has 5 friends with an average of 3.20 friends each
Qunisha    has 4 friends with an average of 3.25 friends each
Ronda      has 2 friends with an average of 3.00 friends each
Sameerah   has 2 friends with an average of 4.50 friends each
Shanise    has 2 friends with an average of 5.00 friends each

People have 1.07 fewer friends than the average number of friends their friends have


### Connectivity

One basic property of graphs is connectivity: is everyone in the graph connected to everyone else?  Let's figure out if our social graph is connected.  (This is Dijkstra's algorithm)

In [21]:
can_reach = dict()

for person in names:
    can_reach[person] = {person:0}
    stack = [person] 
    while stack:
        friend = stack.pop(0) # pop from front to do breadth first search 
        # using set difference to find friends of friend who 
        # we have not already seen (which would make them included in can_reach[person])
        new_friends = set(edges[friend]).difference(can_reach[person])
        # add new friends to stack.
        stack.extend(new_friends)
        # calculate steps to new friends
        steps = can_reach[person][friend] + 1
        # add to can_reach[person] via dict.update and dict comprehension
        can_reach[person].update({f:steps for f in new_friends})

In [23]:
for k,v in can_reach.items():
    print(f'{k:10} can reach {len(v)}/{n}')

{'Cade': {'Cade': 0, 'Moises': 1, 'Joseangel': 2, 'Christiaan': 2, 'Shanise': 2, 'Qunisha': 2, 'Sameerah': 3, 'Camry': 3, 'Ronda': 3, 'Kaetlyn': 3}, 'Camry': {'Camry': 0, 'Christiaan': 1, 'Sameerah': 2, 'Joseangel': 2, 'Shanise': 2, 'Moises': 2, 'Qunisha': 3, 'Cade': 3, 'Ronda': 4, 'Kaetlyn': 4}, 'Christiaan': {'Christiaan': 0, 'Sameerah': 1, 'Joseangel': 1, 'Moises': 1, 'Camry': 1, 'Shanise': 1, 'Qunisha': 2, 'Cade': 2, 'Ronda': 3, 'Kaetlyn': 3}, 'Joseangel': {'Joseangel': 0, 'Sameerah': 1, 'Christiaan': 1, 'Moises': 1, 'Qunisha': 1, 'Shanise': 2, 'Camry': 2, 'Cade': 2, 'Ronda': 2, 'Kaetlyn': 2}, 'Kaetlyn': {'Kaetlyn': 0, 'Ronda': 1, 'Qunisha': 1, 'Joseangel': 2, 'Moises': 2, 'Sameerah': 3, 'Christiaan': 3, 'Cade': 3, 'Shanise': 3, 'Camry': 4}, 'Moises': {'Moises': 0, 'Cade': 1, 'Joseangel': 1, 'Christiaan': 1, 'Shanise': 1, 'Qunisha': 1, 'Sameerah': 2, 'Camry': 2, 'Ronda': 2, 'Kaetlyn': 2}, 'Qunisha': {'Qunisha': 0, 'Joseangel': 1, 'Ronda': 1, 'Moises': 1, 'Kaetlyn': 1, 'Sameerah': 2

In [217]:
for k,v in can_reach.items():
    print(f'{k:10}', v)

Adelia     {'Adelia': 0, 'Hogan': 1, 'Marjorie': 1, 'Porschia': 1, 'Sabreena': 1, 'Mazin': 1, 'Naim': 1, 'Brenner': 1, 'Tamilia': 1, 'Patric': 1, 'Jeanell': 1, 'Susanne': 1, 'Valine': 1, 'Venetta': 1, 'Laria': 2, 'Raina': 2, 'Wafa': 2, 'Ted': 2, 'Kaylor': 2, 'Chrystine': 2, 'Jonika': 2, 'Arley': 2, 'Simone': 2, 'Archie': 2, 'Lakeita': 2, 'Leigh': 2, 'Tinesha': 2, 'Romney': 2, 'Aser': 2, 'Kizzie': 2, 'Krystyn': 2, 'Jermika': 2, 'Lasheika': 2, 'Elzabeth': 2, 'Jazzmon': 2, 'Tahra': 2, 'Isidore': 2, 'Lashonda': 2, 'Marleny': 2, 'Sabreen': 2, 'Tahara': 2, 'Ibeth': 2, 'Arlie': 2, 'Heathr': 2, 'Cezar': 2, 'Kezia': 2, 'Mareena': 2, 'Jeston': 2, 'Shikita': 2, 'Ayonna': 2, 'Dung': 2, 'Marnell': 2, 'Alfonzo': 2, 'Courtnay': 2, 'Shareen': 2, 'Lavonte': 2, 'Hunter': 2, 'Natassja': 3, 'Amish': 3, 'Shawta': 3, 'Tynetta': 3, 'Katharine': 3, 'Mickaela': 3, 'Dejan': 3, 'Shaneice': 3, 'Rhiannon': 3, 'Raymund': 3, 'Viana': 3, 'Mccall': 3, 'Mirranda': 3, 'Racquel': 3, 'Branson': 3, 'Jermine': 3, 'Elnora': 

## Small world

Milgram did a neat experiment, which has captivated folks' imagination: we are more collected than we think.

Let's get our expectations squared away: here we have N people, each with an average of $N p$ friends.  $p$ is fairly small, like 0.05.  So we have 100 people, each with about 5 friends, on average.  If we pick a random pair of people, how long is the path between them?

The small world network phenomenon is that for many different types of networks, the average shortest path is proportional to log(n) where n is the number of nodes.

Let's calculate our average shortest path.

In [218]:
n_connected = 0
sum_min_path = 0

for i in range(n-1):
    for j in range(i+1, n):
        if names[j] in can_reach[names[i]]: # path exists from i to j
            sum_min_path += can_reach[names[i]][names[j]] # min path from i to j
            n_connected += 1 

# this is average min path among *connected* people.  disconnected pairs do not contribute.
sum_min_path / n_connected

2.9254725085910653

Such small world phenomena arise in many types of networks.  In fact, to avoid such a property, we must consider networks that are very regular, like a lattice, or a ring.  Such networks do arise, when we consider connections that are more stratified, such as the network defined by the relation "went to high school with", or "shook hands with" (when considering people in the past, and in the future).  

## Clustering / cliquishness

The degree of clustering or cliquishness of a social network amounts to asking whether friends of friends are likely to be friends.  We will calculate this as the proportion of triads that are close.


In [222]:
person_clustering = []
for person,friends in edges.items():
    k = len(friends)
    if k < 2:
        person_clustering += [None]
    else:
        n_triads = 0
        n_possible = 0
        for i in range(k-1):
            for j in range(i+1,k):
                n_possible += 1
                if friends[j] in edges[friends[i]]:
                    n_triads += 1
        person_clustering += [n_triads / n_possible]

print(f'{p=} mean={mean([c for c in person_clustering if c is not None])}')


p=0.050656872045869016 mean=0.044500487435270054


So with the random graph, we have no clustering -- pairs of people who share a friend are no more likely to be friends than average.  

## Different types of network

### lattice networks

The premise of lattice networks is that all people have an underlying location, and people are connected only to those people they are close to.  There are many subtle variations of this: What is the location on?  Most simply, it would be a ring, but it could be a 2d space, or something more complicated.  How does the presence of edges decrease with distance?  Perhaps each node is connected to the closest k nodes?  Perhaps the probability of connection decreases with distance?  etc.  While these are important distinctions, lets not worry about it.

In [243]:
max_dist = 2  # degree = max_dist*2

edges = dict(zip(names, [[] for i in range(n)]))

for i in range(n-1):
    for j in range(i+1,n):
        distance = min((i-j)%n, (j-i)%n)  # distance along ring defined by index from 0 to n-1 (alphabetical order)
        if distance <= max_dist:
            edges[names[i]].append(names[j])
            edges[names[j]] += [names[i]]

In [246]:
for k,v in edges.items():
    print(f'{k:10}', v)

Adelia     ['Alexi', 'Alfonzo', 'Yakov', 'Yaw']
Alexi      ['Adelia', 'Alfonzo', 'Amish', 'Yaw']
Alfonzo    ['Adelia', 'Alexi', 'Amish', 'Archie']
Amish      ['Alexi', 'Alfonzo', 'Archie', 'Arley']
Archie     ['Alfonzo', 'Amish', 'Arley', 'Arlie']
Arley      ['Amish', 'Archie', 'Arlie', 'Aser']
Arlie      ['Archie', 'Arley', 'Aser', 'Ayonna']
Aser       ['Arley', 'Arlie', 'Ayonna', 'Benjamen']
Ayonna     ['Arlie', 'Aser', 'Benjamen', 'Branson']
Benjamen   ['Aser', 'Ayonna', 'Branson', 'Brenner']
Branson    ['Ayonna', 'Benjamen', 'Brenner', 'Yaw']
Brenner    ['Benjamen', 'Branson', 'Brittiney', 'Carine']
Brittiney  ['Brenner', 'Carine', 'Cezar', 'Romney']
Carine     ['Brenner', 'Brittiney', 'Cezar', 'Charita']
Cezar      ['Brittiney', 'Carine', 'Charita', 'Cherylann']
Charita    ['Carine', 'Cezar', 'Cherylann', 'Chrystine']
Cherylann  ['Cezar', 'Charita', 'Chrystine', 'Courtnay']
Chrystine  ['Charita', 'Cherylann', 'Courtnay', 'Dani']
Courtnay   ['Cherylann', 'Chrystine', 'Dani', 'Dejan

### Watts-Strogatz perturbed ring networks

Watts & Strogatz showed that lattice networks (of the sort we defined above) can gain small world properties with a very small number of randomly rewired edges, which create long-distance ties:

In [247]:
n_perturbations = 4

for name in random.sample(names, n_perturbations):
    old_name = random.choice(edges[name])
    new_name = random.choice(names)
    # remove previous edges
    edges[name].remove(old_name)
    edges[old_name].remove(name)
    # add new edges
    edges[name].append(new_name)
    edges[new_name].append(name)

### Barabasi-Albert scale-free networks

Scale free networks are defined by their power-law degree distribution: most nodes have very few edges and a few nodes have very many edges.  This describes lots of social networks, such as twitter followers: most people have very few followers, and a small number of accounts have millions of followers.

Barabasi & Albert described an algorithm for generating such scale-free networks.

This is a preferential attachment model

In [270]:
edges = dict(zip(names, [[] for i in range(n)]))

m0 = 5 # number of initial nodes
m = 2 # new edges per node

# initialize with connected graph of m0 nodes (here... ring lattice)
for i in range(m0):
    j = (i-1) % m0
    edges[names[i]].append(names[j])
    edges[names[j]].append(names[i])

# add new nodes, each with m preferential attachment.
for i in range(m0, n):
    degree = {k:len(v) for k,v in edges.items() if len(v)>0}
    j = 0
    while j < m:
        target = random.choices([x for x in degree], weights=[v for k,v in degree.items()], k = 1)[0]
        edges[names[i]].append(target)
        edges[target].append(names[i])
        degree.pop(target)
        j += 1

        
for k,v in sorted(edges.items(), key=lambda item: -len(item[1])):
    print(f'{k:>10} {len(v)}')



   Alfonzo 21
    Adelia 19
     Amish 17
    Archie 14
     Arley 12
      Dani 12
    Carine 10
      Dung 10
   Branson 9
  Benjamen 8
     Ibeth 8
     Alexi 7
   Brenner 7
   Jeanell 7
     Dejan 6
     Arlie 5
   Charita 5
 Cherylann 5
    Elnora 5
     Hogan 5
   Jermine 5
   Krystyn 5
    Ayonna 4
  Courtnay 4
    Heathr 4
  Lashonda 4
   Lavonte 4
  Marjorie 4
  Marrissa 4
    Romney 4
      Aser 3
     Cezar 3
   Isidore 3
   Jazzmon 3
    Jeston 3
Johndaniel 3
 Katharine 3
   Lakeita 3
  Lasheika 3
      Noha 3
   Pearlie 3
   Racquel 3
   Raymund 3
  Rhiannon 3
   Sabreen 3
    Simone 3
 Brittiney 2
 Chrystine 2
  Elzabeth 2
     Garth 2
    Hunter 2
   Jermika 2
    Jonika 2
   Karisha 2
    Kaylor 2
     Kezia 2
      Kiah 2
    Kizzie 2
     Laria 2
     Laysa 2
     Leigh 2
   Mareena 2
   Marleny 2
   Marnell 2
     Mazin 2
    Mccall 2
  Mickaela 2
  Mirranda 2
  Mystique 2
      Naim 2
    Nashea 2
  Natassja 2
  Nikolaus 2
    Patric 2
  Porschia 2
     Raina 2
  Ra

In [256]:
degree.keys()


[dict_keys(['Adelia', 'Alexi', 'Alfonzo', 'Amish', 'Archie'])]